## Reload the `EC_estimator.py` code to consider any changes

In [5]:
#reload model magic
%load_ext autoreload
%autoreload 2

## Import X2_estimator module

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import X2_estimator as annx2
import tensorflow as tf

In [7]:
tf.__version__

'2.2.0'

## Multi-Study X2_DIS Training

Train X2_DIS model based on multiple studies: `data_x2_base.csv`, `data_x2_base_ns.csv`, and `data_x2_6k_tunnel.csv`

In [ ]:
# Train X2_DIS model based on multiple studies
# This cell trains a single X2_DIS model using combined data from all studies
# Uses EXACT same training pattern as EC_estimator (118-day window with antecedent extraction)

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import X2_estimator as annx2
import tensorflow as tf

# ------------------------------------------------------------
# Settings - MATCHING EC EXACTLY
# ------------------------------------------------------------
predictors = ["NDOI", "smscg", "tide"]  # X2 has 3 predictors (EC has 7)
target = "X2_DIS"

dataset_files = [
    "Inputs/data_x2_base.csv",
    "Inputs/data_x2_base_ns.csv",
    "Inputs/data_x2_6k_tunnel.csv"
]

# Use 118-day window like EC (not 18!)
# The 118 days will be converted to 18 antecedent features:
#   - 1 current day
#   - 7 daily values  
#   - 10 averaged blocks of 11 days each
WINDOW = 118  

test_start = "1923-01-01"
test_end = "1939-12-31"
train_start = "1940-01-01"
train_end = "2015-12-31"
calib_prop = 0.8

# ------------------------------------------------------------
# Helper functions (identical to EC)
# ------------------------------------------------------------
def load_and_preprocess_dataset(filepath):
    """Load and preprocess a single dataset file."""
    df = pd.read_csv(filepath, index_col=0)
    df = df.reset_index()
    df.columns = ["date"] + list(df.columns[1:])
    df["date"] = pd.to_datetime(df["date"], errors="raise")
    df = df.sort_values("date")
    df = df.reset_index(drop=True)
    return df

def make_windows(df_raw, predictors, target, window):
    """Create sliding windows of input features and corresponding targets."""
    X_all = df_raw[predictors].to_numpy()
    y_all = df_raw[target].to_numpy().reshape(-1, 1)
    if len(df_raw) < window:
        raise ValueError(f"Not enough rows ({len(df_raw)}) to build a window of {window}.")
    Xw, yw = [], []
    for t in range(window - 1, len(df_raw)):
        Xw.append(X_all[t - window + 1 : t + 1, :])
        yw.append(y_all[t])
    return np.stack(Xw, axis=0), np.vstack(yw)

# ------------------------------------------------------------
# Load datasets once
# ------------------------------------------------------------
print("Loading X2 datasets...")
datasets = {fp.split("/")[-1].replace(".csv", ""): load_and_preprocess_dataset(fp) for fp in dataset_files}

datasets_test_raw = {}
datasets_train_full_raw = {}
for name, df in datasets.items():
    datasets_test_raw[name] = df[(df["date"] >= test_start) & (df["date"] <= test_end)].copy()
    datasets_train_full_raw[name] = df[(df["date"] >= train_start) & (df["date"] <= train_end)].copy()
    print(f"  {name}: Train={len(datasets_train_full_raw[name])}, Test={len(datasets_test_raw[name])}")

# Combine all training data for preprocessing layer adaptation
df_train_full_combined = pd.concat([datasets_train_full_raw[name] for name in datasets.keys()], ignore_index=True)

# ------------------------------------------------------------
# Build windows and combine datasets
# ------------------------------------------------------------
print(f"\n{'='*80}")
print(f"TRAINING X2_DIS MODEL FROM MULTIPLE STUDIES")
print(f"Window size: {WINDOW} days -> 18 antecedent features (like EC)")
print(f"{'='*80}")

np.random.seed(42)  # For reproducibility

X_train_combined, y_train_combined = [], []
X_val_combined, y_val_combined = [], []
X_test_combined, y_test_combined = [], []

# Track dates for monthly aggregation
train_dates_combined = []
val_dates_combined = []
test_dates_combined = []

for name in datasets.keys():
    print(f"\nProcessing dataset: {name}")
    df_train_full_raw = datasets_train_full_raw[name]
    df_test_raw = datasets_test_raw[name]
    
    X_train_full_seq, y_train_full_arr = make_windows(df_train_full_raw, predictors, target, WINDOW)
    X_test_seq, y_test_arr = make_windows(df_test_raw, predictors, target, WINDOW)
    
    n_train_samples = X_train_full_seq.shape[0]
    vset = np.random.rand(n_train_samples)
    train_mask = vset < calib_prop
    val_mask = vset >= calib_prop
    
    X_train_combined.append(X_train_full_seq[train_mask])
    y_train_combined.append(y_train_full_arr[train_mask])
    X_val_combined.append(X_train_full_seq[val_mask])
    y_val_combined.append(y_train_full_arr[val_mask])
    X_test_combined.append(X_test_seq)
    y_test_combined.append(y_test_arr)
    
    # Get dates for windows (window ends at each date)
    train_full_dates = df_train_full_raw["date"].iloc[WINDOW-1:].reset_index(drop=True)
    test_window_dates = df_test_raw["date"].iloc[WINDOW-1:].reset_index(drop=True)
    
    # Append dates for monthly aggregation
    train_dates_combined.append(train_full_dates[train_mask].reset_index(drop=True))
    val_dates_combined.append(train_full_dates[val_mask].reset_index(drop=True))
    test_dates_combined.append(test_window_dates)
    
    print(f"  Train samples: {sum(train_mask)}, Val samples: {sum(val_mask)}, Test samples: {len(X_test_seq)}")

# Concatenate all datasets
X_train_seq = np.concatenate(X_train_combined, axis=0)
y_train_arr = np.concatenate(y_train_combined, axis=0)
X_val_seq = np.concatenate(X_val_combined, axis=0)
y_val_arr = np.concatenate(y_val_combined, axis=0)
X_test_seq = np.concatenate(X_test_combined, axis=0)
y_test_arr = np.concatenate(y_test_combined, axis=0)

# Combine dates
train_dates_all = pd.concat(train_dates_combined, ignore_index=True)
val_dates_all = pd.concat(val_dates_combined, ignore_index=True)
test_dates_all = pd.concat(test_dates_combined, ignore_index=True)

print(f"\nTotal Train: {X_train_seq.shape[0]}, Val: {X_val_seq.shape[0]}, Test: {X_test_seq.shape[0]}")

# ------------------------------------------------------------
# Format inputs for model - Each is (N, 118)
# ------------------------------------------------------------
X_train = [X_train_seq[:, :, i] for i in range(len(predictors))]
X_val = [X_val_seq[:, :, i] for i in range(len(predictors))]
X_test = [X_test_seq[:, :, i] for i in range(len(predictors))]

y_train = pd.DataFrame(y_train_arr, columns=[target])
y_val = pd.DataFrame(y_val_arr, columns=[target])
y_test = pd.DataFrame(y_test_arr, columns=[target])

print(f"\nX_train shapes: {[x.shape for x in X_train]}")  # expect 3 × (N, 118)
print(f"y_train range: [{y_train.values.min():.1f}, {y_train.values.max():.1f}]")

# ------------------------------------------------------------
# Build model - EXACTLY MATCHING EC APPROACH
# Uses annx2.build_model_inputs() and annx2.preprocessing_layers()
# ------------------------------------------------------------
inputs = annx2.build_model_inputs(None)

# Preprocessing layers: raw 118-day input -> 18 antecedents -> z-score normalization
layers = annx2.preprocessing_layers(df_train_full_combined, inputs, X_train)

print(f"\nBuilt {len(layers)} input preprocessing layers")
print(f"  Each layer: 118 days -> 18 antecedents -> z-score normalized")

# Build model - stores scaling parameters for inference
model, tensorboard_cb = annx2.build_model(layers, inputs, y_train=y_train)

model.summary()

# ------------------------------------------------------------
# Scale target and train model - MATCHING EC APPROACH
# ------------------------------------------------------------
y_scaler = MinMaxScaler(feature_range=(0.1, 0.9))
y_train_scaled = y_scaler.fit_transform(y_train.values)
y_val_scaled = y_scaler.transform(y_val.values)
y_test_scaled = y_scaler.transform(y_test.values)

print(f"\nScaled y_train range: [{y_train_scaled.min():.4f}, {y_train_scaled.max():.4f}]")
print(f"Scaled y_val range  : [{y_val_scaled.min():.4f}, {y_val_scaled.max():.4f}]")

print("\nTraining X2_DIS model (EC-style approach)...")
print(f"  Optimizer: Adam, Loss: MSE, Output: Linear")
print(f"  Epochs: 3000, Patience: 1000, Batch size: 128")

history, model = annx2.train_model(
    model, tensorboard_cb,
    X_train, y_train_scaled,
    X_val, y_val_scaled,
    epochs=3000, patience=1000, batch_size=128
)

# ------------------------------------------------------------
# Create inference model and evaluate (using EC pattern)
# ------------------------------------------------------------
inference_model = annx2.create_inference_model(model)

# Make predictions (unscaled, in original X2 units - km)
y_train_pred = inference_model.predict(X_train, verbose=0)
y_val_pred = inference_model.predict(X_val, verbose=0)
y_test_pred = inference_model.predict(X_test, verbose=0)

print(f"\nPrediction ranges (original X2 km units):")
print(f"  y_train_pred: [{y_train_pred.min():.1f}, {y_train_pred.max():.1f}]")
print(f"  y_val_pred  : [{y_val_pred.min():.1f}, {y_val_pred.max():.1f}]")
print(f"  y_test_pred : [{y_test_pred.min():.1f}, {y_test_pred.max():.1f}]")

# ------------------------------------------------------------
# Calculate DAILY metrics
# ------------------------------------------------------------
train_rmse = np.sqrt(mean_squared_error(y_train.values, y_train_pred))
train_r2 = r2_score(y_train.values, y_train_pred)
val_rmse = np.sqrt(mean_squared_error(y_val.values, y_val_pred))
val_r2 = r2_score(y_val.values, y_val_pred)
test_rmse = np.sqrt(mean_squared_error(y_test.values, y_test_pred))
test_r2 = r2_score(y_test.values, y_test_pred)

# ------------------------------------------------------------
# Calculate MONTHLY metrics
# ------------------------------------------------------------
y_train_with_dates = pd.DataFrame(y_train.values, index=pd.to_datetime(train_dates_all), columns=[target])
y_train_pred_df = pd.DataFrame(y_train_pred, index=pd.to_datetime(train_dates_all), columns=['pred'])
y_val_with_dates = pd.DataFrame(y_val.values, index=pd.to_datetime(val_dates_all), columns=[target])
y_val_pred_df = pd.DataFrame(y_val_pred, index=pd.to_datetime(val_dates_all), columns=['pred'])
y_test_with_dates = pd.DataFrame(y_test.values, index=pd.to_datetime(test_dates_all), columns=[target])
y_test_pred_df = pd.DataFrame(y_test_pred, index=pd.to_datetime(test_dates_all), columns=['pred'])

# Monthly aggregation
y_train_monthly = y_train_with_dates.resample('M').mean()
y_train_pred_monthly = y_train_pred_df.resample('M').mean()
y_val_monthly = y_val_with_dates.resample('M').mean()
y_val_pred_monthly = y_val_pred_df.resample('M').mean()
y_test_monthly = y_test_with_dates.resample('M').mean()
y_test_pred_monthly = y_test_pred_df.resample('M').mean()

# Compute monthly metrics
train_rmse_monthly = np.sqrt(mean_squared_error(y_train_monthly.values, y_train_pred_monthly.values))
train_r2_monthly = r2_score(y_train_monthly.values, y_train_pred_monthly.values)
val_rmse_monthly = np.sqrt(mean_squared_error(y_val_monthly.values, y_val_pred_monthly.values))
val_r2_monthly = r2_score(y_val_monthly.values, y_val_pred_monthly.values)
test_rmse_monthly = np.sqrt(mean_squared_error(y_test_monthly.values, y_test_pred_monthly.values))
test_r2_monthly = r2_score(y_test_monthly.values, y_test_pred_monthly.values)

# ------------------------------------------------------------
# Print results
# ------------------------------------------------------------
print(f"\n{'='*80}")
print(f"X2_DIS MODEL RESULTS (Multi-Study Training - EC-style with 118-day window)")
print(f"{'='*80}")
print(f"\n--- DAILY METRICS ---")
print(f"  Train - RMSE: {train_rmse:.2f} km, R²: {train_r2:.4f}")
print(f"  Val   - RMSE: {val_rmse:.2f} km, R²: {val_r2:.4f}")
print(f"  Test  - RMSE: {test_rmse:.2f} km, R²: {test_r2:.4f}")
print(f"\n--- MONTHLY METRICS ---")
print(f"  Train - RMSE: {train_rmse_monthly:.2f} km, R²: {train_r2_monthly:.4f} ({len(y_train_monthly)} months)")
print(f"  Val   - RMSE: {val_rmse_monthly:.2f} km, R²: {val_r2_monthly:.4f} ({len(y_val_monthly)} months)")
print(f"  Test  - RMSE: {test_rmse_monthly:.2f} km, R²: {test_r2_monthly:.4f} ({len(y_test_monthly)} months)")

# Store results
results = {
    'Model': 'X2_DIS',
    'Train_Samples': len(y_train),
    'Val_Samples': len(y_val),
    'Test_Samples': len(y_test),
    # Daily metrics
    'Train_RMSE_Daily': train_rmse,
    'Train_R2_Daily': train_r2,
    'Val_RMSE_Daily': val_rmse,
    'Val_R2_Daily': val_r2,
    'Test_RMSE_Daily': test_rmse,
    'Test_R2_Daily': test_r2,
    # Monthly metrics
    'Train_Months': len(y_train_monthly),
    'Val_Months': len(y_val_monthly),
    'Test_Months': len(y_test_monthly),
    'Train_RMSE_Monthly': train_rmse_monthly,
    'Train_R2_Monthly': train_r2_monthly,
    'Val_RMSE_Monthly': val_rmse_monthly,
    'Val_R2_Monthly': val_r2_monthly,
    'Test_RMSE_Monthly': test_rmse_monthly,
    'Test_R2_Monthly': test_r2_monthly,
    # Value ranges
    'y_train_min': y_train.values.min(),
    'y_train_max': y_train.values.max(),
    'y_test_min': y_test.values.min(),
    'y_test_max': y_test.values.max()
}

results_df = pd.DataFrame([results])

# ------------------------------------------------------------
# Save models
# ------------------------------------------------------------
model_save_path = './Export/X2_DIS'
os.makedirs(model_save_path, exist_ok=True)
annx2.save_model(model, model_save_path)
inference_model.save(os.path.join(model_save_path, 'inference_model'))

# Save results to CSV
results_df.to_csv("Export/x2_multi_study_results.csv", index=False)

print(f"\n✓ Model saved to: {model_save_path}")
print(f"✓ Inference model saved to: {model_save_path}/inference_model")
print(f"✓ Results saved to: Export/x2_multi_study_results.csv")

# Clear session to free memory
tf.keras.backend.clear_session()
print("\n✓ Training complete. Session cleared.")

Loading X2 datasets...
  data_x2_base: Train=27576, Test=6209
  data_x2_base_ns: Train=27576, Test=6209
  data_x2_6k_tunnel: Train=27576, Test=6209

TRAINING X2_DIS MODEL FROM MULTIPLE STUDIES
Window size: 118 days -> 18 antecedent features (like EC)

Processing dataset: data_x2_base
  Train samples: 21969, Val samples: 5490, Test samples: 6092

Processing dataset: data_x2_base_ns
  Train samples: 22034, Val samples: 5425, Test samples: 6092

Processing dataset: data_x2_6k_tunnel
  Train samples: 21952, Val samples: 5507, Test samples: 6092

Total Train: 65955, Val: 16422, Test: 18276

X_train shapes: [(65955, 118), (65955, 118), (65955, 118)]
y_train range: [47.8, 96.7]

Built 3 input preprocessing layers
  Each layer: 118 days -> 18 antecedents -> z-score normalized
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
NDOI_inp

## Evaluation of the model

In [ ]:
# Calculate metrics for ANN
# The trained model outputs scaled predictions [0.1, 0.9]
# We create an inference model that automatically inverse-transforms to original units

metrics = []

# Predict with trained model (outputs are scaled [0.1, 0.9])
y_train_pred_scaled = model.predict(X_train, verbose=0)
y_val_pred_scaled = model.predict(X_val, verbose=0)
y_test_pred_scaled = model.predict(X_test, verbose=0)

# Create inference model with automatic inverse scaling
inference_model = annx2.create_inference_model(model)

# Predict with inference model (outputs are original units)
y_train_pred = inference_model.predict(X_train, verbose=0)
y_val_pred = inference_model.predict(X_val, verbose=0)
y_test_pred = inference_model.predict(X_test, verbose=0)

print("=== Predictions (original X2 km units, last 3 samples) ===")
print("train:", y_train_pred[-3:].ravel())
print("val  :", y_val_pred[-3:].ravel())
print("test :", y_test_pred[-3:].ravel())

print("\n=== Scaled predictions range ===")
print(f"y_train_pred_scaled: [{y_train_pred_scaled.min():.4f}, {y_train_pred_scaled.max():.4f}]")
print(f"y_val_pred_scaled  : [{y_val_pred_scaled.min():.4f}, {y_val_pred_scaled.max():.4f}]")
print(f"y_test_pred_scaled : [{y_test_pred_scaled.min():.4f}, {y_test_pred_scaled.max():.4f}]")

print("\n=== Original unit predictions range ===")
print(f"y_train_pred: [{y_train_pred.min():.1f}, {y_train_pred.max():.1f}]")
print(f"y_val_pred  : [{y_val_pred.min():.1f}, {y_val_pred.max():.1f}]")
print(f"y_test_pred : [{y_test_pred.min():.1f}, {y_test_pred.max():.1f}]")

# Compute metrics using original (unscaled) values
y_train_true = y_train.values  # Original X2 values
y_val_true = y_val.values      # Original X2 values
y_test_true = y_test.values    # Original X2 values

metrics.append(
    annx2.calculate_metrics("ANN", y_train, y_train_pred, y_test, y_test_pred)
)

df_metrics = pd.DataFrame(metrics)
print("\n=== Model Performance ===")
print(df_metrics)

# Additional validation metrics
print("\n=== Validation Set Metrics ===")
val_rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred))
val_r2 = r2_score(y_val_true, y_val_pred)
print(f"Validation RMSE: {val_rmse:.2f}")
print(f"Validation R²  : {val_r2:.4f}")

NameError: name 'model' is not defined

In [4]:
annx2.plot_history(history)

NameError: name 'annx2' is not defined

## Plot the ANN prediction versus the test target as a time series

In [ ]:
# Plot predictions vs actual (in original X2 km units)
import matplotlib.pyplot as plt

ax = y_test.plot(label='target')
pred = pd.DataFrame(index=y_test.index, data=y_test_pred, columns=['prediction'])
print(f"Last prediction: {pred.iloc[-1].values[0]:.1f}")
pred.plot(ax=ax)
plt.ylabel('X2 (km)')
plt.legend(["target", "ann"])
plt.title('Test Set: ANN Prediction vs Target')
plt.show()

## Target vs ANN predictions scatter plot (Test Data)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Calculate R² for test data
y_test_np = y_test.values.ravel()
y_test_pred_np = y_test_pred.ravel()
r2_test = r2_score(y_test_np, y_test_pred_np)

# Create scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(y_test_pred_np, y_test_np, alpha=0.5, s=10)

# Add diagonal line (perfect prediction)
min_val = min(y_test_np.min(), y_test_pred_np.min())
max_val = max(y_test_np.max(), y_test_pred_np.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

# Add R² text to plot
plt.text(0.05, 0.95, f'R² = {r2_test:.3f}', transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.xlabel('ANN Predictions', fontsize=12)
plt.ylabel('Target Values', fontsize=12)
plt.title('Target vs ANN Predictions (Test Data)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Training Data: Target vs ANN predictions scatter plot

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Calculate R² for training data
y_train_np = y_train.values.ravel()
y_train_pred_np = y_train_pred.ravel()
r2_train = r2_score(y_train_np, y_train_pred_np)

# Create scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(y_train_pred_np, y_train_np, alpha=0.5, s=10)

# Add diagonal line (perfect prediction)
min_val = min(y_train_np.min(), y_train_pred_np.min())
max_val = max(y_train_np.max(), y_train_pred_np.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

# Add R² text to plot
plt.text(0.05, 0.95, f'R² = {r2_train:.3f}', transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.xlabel('ANN Predictions', fontsize=12)
plt.ylabel('Target Values', fontsize=12)
plt.title('Target vs ANN Predictions (Training Data)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Monthly aggregated data analysis

In [ ]:
# Add date indices to DataFrames for monthly aggregation
# Dates were tracked during dataset combination in the training cell
# train_dates_all, val_dates_all, test_dates_all are combined from all 3 datasets

# Set date indices on target DataFrames
y_train_with_dates = y_train.copy()
y_train_with_dates.index = pd.to_datetime(train_dates_all)
y_test_with_dates = y_test.copy()
y_test_with_dates.index = pd.to_datetime(test_dates_all)

# Create DataFrames with predictions and date indices
y_train_pred_df = pd.DataFrame(index=pd.to_datetime(train_dates_all), data=y_train_pred, columns=['x2_pred'])
y_test_pred_df = pd.DataFrame(index=pd.to_datetime(test_dates_all), data=y_test_pred, columns=['x2_pred'])

# Monthly aggregation - using mean
y_train_monthly = y_train_with_dates.resample('M').mean()
y_train_pred_monthly = y_train_pred_df.resample('M').mean()

y_test_monthly = y_test_with_dates.resample('M').mean()
y_test_pred_monthly = y_test_pred_df.resample('M').mean()

print("Training Data - Monthly Averages")
print(f"Number of months: {len(y_train_monthly)}")
print(f"Date range: {y_train_monthly.index.min().date()} to {y_train_monthly.index.max().date()}")
print(y_train_monthly.head())

print("\nTest Data - Monthly Averages")
print(f"Number of months: {len(y_test_monthly)}")
print(f"Date range: {y_test_monthly.index.min().date()} to {y_test_monthly.index.max().date()}")
print(y_test_monthly.head())

## Monthly Training Data: Time Series Plot

In [ ]:
import matplotlib.pyplot as plt

ax = y_train_monthly.plot(figsize=(12, 5))
y_train_pred_monthly.plot(ax=ax)
plt.legend(["target", "ann"])
plt.title('Training Data: Target vs ANN Predictions (Monthly Averages)')
plt.xlabel('Date')
plt.ylabel('X2 (km) (Monthly Average)')
plt.grid(True, alpha=0.3)
plt.show()

## Monthly Training Data: Scatter Plot with R²

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Calculate R² for monthly training data
y_train_monthly_np = y_train_monthly.values.ravel()
y_train_pred_monthly_np = y_train_pred_monthly.values.ravel()
r2_train_monthly = r2_score(y_train_monthly_np, y_train_pred_monthly_np)

# Create scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(y_train_pred_monthly_np, y_train_monthly_np, alpha=0.6, s=50)

# Add diagonal line (perfect prediction)
min_val = min(y_train_monthly_np.min(), y_train_pred_monthly_np.min())
max_val = max(y_train_monthly_np.max(), y_train_pred_monthly_np.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

# Add R² text to plot
plt.text(0.05, 0.95, f'R² = {r2_train_monthly:.3f}', transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.xlabel('ANN Predictions (Monthly Average)', fontsize=12)
plt.ylabel('Target Values (Monthly Average)', fontsize=12)
plt.title('Target vs ANN Predictions (Training Data - Monthly)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Monthly Test Data: Time Series Plot

In [ ]:
import matplotlib.pyplot as plt

ax = y_test_monthly.plot(figsize=(12, 5))
y_test_pred_monthly.plot(ax=ax)
plt.legend(["target", "ann"])
plt.title('Test Data: Target vs ANN Predictions (Monthly Averages)')
plt.xlabel('Date')
plt.ylabel('X2 (km) (Monthly Average)')
plt.grid(True, alpha=0.3)
plt.show()

## Monthly Test Data: Scatter Plot with R²

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Calculate R² for monthly test data
y_test_monthly_np = y_test_monthly.values.ravel()
y_test_pred_monthly_np = y_test_pred_monthly.values.ravel()
r2_test_monthly = r2_score(y_test_monthly_np, y_test_pred_monthly_np)

# Create scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(y_test_pred_monthly_np, y_test_monthly_np, alpha=0.6, s=50)

# Add diagonal line (perfect prediction)
min_val = min(y_test_monthly_np.min(), y_test_pred_monthly_np.min())
max_val = max(y_test_monthly_np.max(), y_test_pred_monthly_np.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

# Add R² text to plot
plt.text(0.05, 0.95, f'R² = {r2_test_monthly:.3f}', transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.xlabel('ANN Predictions (Monthly Average)', fontsize=12)
plt.ylabel('Target Values (Monthly Average)', fontsize=12)
plt.title('Target vs ANN Predictions (Test Data - Monthly)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Save the final model

In [ ]:
# Create station-specific save path
model_save_path = './Export/X2_DIS'
os.makedirs(model_save_path, exist_ok=True)

annx2.save_model(model, model_save_path)

# Also save the inference model (with inverse scaling built-in)
inference_model_path = os.path.join(model_save_path, 'inference_model')
inference_model.save(inference_model_path)

print(f"Training model (scaled output) saved at: {model_save_path}")
print(f"Inference model (original units) saved at: {inference_model_path}")
print(f"\nTarget: X2_DIS")
print("\nNote: All scaling layers are now built into the TensorFlow model!")
print("- Output scaling [0.1, 0.9] is applied to X2 predictions during training")
print("- Inference model automatically inverse-scales outputs to original X2 km units")

In [ ]:
# Load saved models with custom layers
custom_objects = {
    'MinMaxScaleLayer': annx2.MinMaxScaleLayer,
    'InverseMinMaxScaleLayer': annx2.InverseMinMaxScaleLayer
}

# Load training model (outputs scaled [0.1, 0.9])
loaded_model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects)

# Load inference model (outputs original units)
inference_model_path = os.path.join(model_save_path, 'inference_model')
loaded_inference_model = tf.keras.models.load_model(inference_model_path, custom_objects=custom_objects)

# Calculate metrics using inference model (original units)
metrics = []

# Predict with scaled model
y_train_pred_scaled = loaded_model.predict(X_train, verbose=0)
y_test_pred_scaled = loaded_model.predict(X_test, verbose=0)

# Predict with inference model (original units)
y_train_pred = loaded_inference_model.predict(X_train, verbose=0)
y_test_pred = loaded_inference_model.predict(X_test, verbose=0)

print("=== Loaded Models Evaluation ===")
print(f"Training model output range (scaled):  [{y_train_pred_scaled.min():.4f}, {y_train_pred_scaled.max():.4f}]")
print(f"Inference model output range (original): [{y_train_pred.min():.1f}, {y_train_pred.max():.1f}]")

metrics.append(annx2.calculate_metrics('ANN (Loaded)', y_train, y_train_pred, y_test, y_test_pred))

df_metrics = pd.DataFrame(metrics)
print(df_metrics)